[View in Colaboratory](https://colab.research.google.com/github/Srinivasankrishnan27/MW/blob/master/age_regression.ipynb)

In [0]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Activation, Dense, Dropout, Flatten
from keras import backend as K

In [3]:
from google.colab import files
uploaded = files.upload()

Saving processed.csv to processed.csv


In [5]:
uploaded = files.upload()

Saving processed_target.csv to processed_target.csv


In [0]:
m = pd.read_csv('processed.csv')
m_target = pd.read_csv('processed_target.csv', header=None)

In [7]:
print(len(m))
print(len(m_target))

178665
178665


In [8]:
m.columns

Index(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '3', '4', '5', '6', '7', '8', '9',
       'ANDROID-BOOKS_AND_REFERENCE', 'ANDROID-COMMUNICATION',
       'ANDROID-DATING', 'ANDROID-ENTERTAINMENT', 'ANDROID-GAME_ACTION',
       'ANDROID-GAME_ADVENTURE', 'ANDROID-GAME_ARCADE', 'ANDROID-GAME_BOARD',
       'ANDROID-GAME_CARD', 'ANDROID-GAME_CASINO', 'ANDROID-GAME_CASUAL',
       'ANDROID-GAME_MUSIC', 'ANDROID-GAME_PUZZLE', 'ANDROID-GAME_RACING',
       'ANDROID-GAME_ROLE_PLAYING', 'ANDROID-GAME_SIMULATION',
       'ANDROID-GAME_SPORTS', 'ANDROID-GAME_STRATEGY', 'ANDROID-GAME_TRIVIA',
       'ANDROID-GAME_WORD', 'ANDROID-HEALTH_AND_FITNESS', 'ANDROID-LIFESTYLE',
       'ANDROID-MAPS_AND_NAVIGATION', 'ANDROID-MUSIC_AND_AUDIO',
       'ANDROID-NEWS_AND_MAGAZINES', 'ANDROID-PERSONALIZATION',
       'ANDROID-PHOTOGRAPHY', 'ANDROID-PRODUCTIVITY', 'ANDROID-SOCIAL',
       'ANDROID-SPORTS', 'ANDROID-TOOLS', 'ANDROID-TRAVEL_AND_LOCAL'

In [0]:
X = m
Y = m_target

In [10]:
input_len = len(X.columns)
print(input_len)

73


In [0]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )
  
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [12]:
# Model Params
batch_size = 100
epochs = 100

# create baseline_model
model = Sequential()
model.add(Dense(80, input_dim=input_len, kernel_initializer='RandomNormal', activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
model.compile(loss=['mape'], optimizer='adam', metrics=[coeff_determination,root_mean_squared_error])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                5920      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 10,001
Trainable params: 10,001
Non-trainable params: 0
_________________________________________________________________


In [29]:
early_stopping = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint(filepath=r'age_prediction.hdf5', 
                               verbose=1, 
                               save_best_only=True)

model.fit(X, Y, 
          batch_size=batch_size, 
          epochs = epochs, 
          verbose=1,
          shuffle=True,
          validation_split=0.1,
         callbacks=[early_stopping, checkpointer])

Train on 160798 samples, validate on 17867 samples
Epoch 1/100
160798/160798 [==============================] - 12s 73us/step - loss: 45.3582 - coeff_determination: -2.7480 - root_mean_squared_error: 13.5884 - val_loss: 29.0453 - val_coeff_determination: -0.5846 - val_root_mean_squared_error: 9.2588

Epoch 00001: val_loss improved from inf to 29.04533, saving model to age_prediction.hdf5
Epoch 2/100
 80000/160798 [=============>................] - ETA: 5s - loss: 30.1237 - coeff_determination: -0.7661 - root_mean_squared_error: 9.5207

160798/160798 [==============================] - 12s 74us/step - loss: 28.8598 - coeff_determination: -0.6726 - root_mean_squared_error: 9.1748 - val_loss: 27.9466 - val_coeff_determination: -0.6463 - val_root_mean_squared_error: 8.9961

Epoch 00002: val_loss improved from 29.04533 to 27.94662, saving model to age_prediction.hdf5
Epoch 3/100
133000/160798 [=======================>......] - ETA: 1s - loss: 25.8297 - coeff_determination: -0.4337 - root_mean_squared_error: 8.3665

160798/160798 [==============================] - 12s 73us/step - loss: 25.7784 - coeff_determination: -0.4328 - root_mean_squared_error: 8.3520 - val_loss: 26.8269 - val_coeff_determination: -0.6486 - val_root_mean_squared_error: 9.1574

Epoch 00003: val_loss improved from 27.94662 to 26.82685, saving model to age_prediction.hdf5
Epoch 4/100
146900/160798 [==========================>...] - ETA: 0s - loss: 23.9125 - coeff_determination: -0.3065 - root_mean_squared_error: 7.8602

160798/160798 [==============================] - 12s 74us/step - loss: 23.8424 - coeff_determination: -0.3017 - root_mean_squared_error: 7.8426 - val_loss: 28.5877 - val_coeff_determination: -0.4561 - val_root_mean_squared_error: 8.4903

Epoch 00004: val_loss did not improve from 26.82685
Epoch 5/100
160798/160798 [==============================] - 12s 73us/step - loss: 22.8678 - coeff_determination: -0.2436 - root_mean_squared_error: 7.5837 - val_loss: 21.9703 - val_coeff_determination: -0.2453 - val_root_mean_squared_error: 7.5621

Epoch 00005: val_loss improved from 26.82685 to 21.97026, saving model to age_prediction.hdf5
Epoch 6/100
  2400/160798 [..............................] - ETA: 11s - loss: 21.5003 - coeff_determination: -0.1601 - root_mean_squared_error: 7.2588

160798/160798 [==============================] - 12s 73us/step - loss: 22.0507 - coeff_determination: -0.1970 - root_mean_squared_error: 7.3805 - val_loss: 21.3558 - val_coeff_determination: -0.1650 - val_root_mean_squared_error: 7.3042

Epoch 00006: val_loss improved from 21.97026 to 21.35584, saving model to age_prediction.hdf5
Epoch 7/100
110900/160798 [===================>..........] - ETA: 3s - loss: 21.7516 - coeff_determination: -0.1830 - root_mean_squared_error: 7.3012

...

160798/160798 [==============================] - 12s 75us/step - loss: 19.8370 - coeff_determination: -0.0477 - root_mean_squared_error: 6.6799 - val_loss: 19.9478 - val_coeff_determination: -0.1196 - val_root_mean_squared_error: 6.9373

Epoch 00050: val_loss did not improve from 19.73770
Epoch 51/100
155000/160798 [===========================>..] - ETA: 0s - loss: 19.8122 - coeff_determination: -0.0486 - root_mean_squared_error: 6.6716

160798/160798 [==============================] - 12s 74us/step - loss: 19.8109 - coeff_determination: -0.0480 - root_mean_squared_error: 6.6715 - val_loss: 20.0331 - val_coeff_determination: 0.0225 - val_root_mean_squared_error: 6.5810

Epoch 00051: val_loss did not improve from 19.73770
Epoch 52/100
160798/160798 [==============================] - 12s 74us/step - loss: 19.8253 - coeff_determination: -0.0465 - root_mean_squared_error: 6.6736 - val_loss: 20.0490 - val_coeff_determination: -0.1220 - val_root_mean_squared_error: 6.9589

Epoch 00052: val_loss did not improve from 19.73770
Epoch 53/100
  3300/160798 [..............................] - ETA: 11s - loss: 19.9852 - coeff_determination: -0.0531 - root_mean_squared_error: 6.7209

160798/160798 [==============================] - 12s 74us/step - loss: 19.7733 - coeff_determination: -0.0450 - root_mean_squared_error: 6.6601 - val_loss: 19.9382 - val_coeff_determination: -0.0797 - val_root_mean_squared_error: 6.8278

Epoch 00053: val_loss did not improve from 19.73770
Epoch 54/100
119100/160798 [=====================>........] - ETA: 2s - loss: 19.7604 - coeff_determination: -0.0428 - root_mean_squared_error: 6.6571

160798/160798 [==============================] - 12s 74us/step - loss: 19.7505 - coeff_determination: -0.0421 - root_mean_squared_error: 6.6497 - val_loss: 19.7461 - val_coeff_determination: -0.0639 - val_root_mean_squared_error: 6.7649

Epoch 00054: val_loss did not improve from 19.73770
Epoch 55/100
153100/160798 [===========================>..] - ETA: 0s - loss: 19.7361 - coeff_determination: -0.0452 - root_mean_squared_error: 6.6513

160798/160798 [==============================] - 12s 75us/step - loss: 19.7354 - coeff_determination: -0.0445 - root_mean_squared_error: 6.6526 - val_loss: 19.8435 - val_coeff_determination: 0.0074 - val_root_mean_squared_error: 6.5865

Epoch 00055: val_loss did not improve from 19.73770
Epoch 56/100
160798/160798 [==============================] - 12s 75us/step - loss: 19.7210 - coeff_determination: -0.0414 - root_mean_squared_error: 6.6419 - val_loss: 19.8046 - val_coeff_determination: -0.0743 - val_root_mean_squared_error: 6.8019

Epoch 00056: val_loss did not improve from 19.73770
Epoch 00056: early stopping


In [13]:
uploaded = files.upload()

Saving age_prediction.hdf5 to age_prediction.hdf5


In [0]:
model.load_weights(r'age_prediction.hdf5')
y_pred = model.predict(X)

In [23]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(mean_absolute_percentage_error(Y, y_pred))

19.57794442752429


In [0]:
from google.colab import files
files.download('age_prediction.hdf5')